# VLBA conversion guide

In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")

XRADIO version 0.0.36 already installed.


## Download dataset

Original VLBA dataset gotten from https://casaguides.nrao.edu/index.php?title=VLBA_Basic_Phase-referencing_Calibration_and_Imaging.

Reduced the data using:

```Python
mstransform(vis='VLBA_TL016B.ms/',outputvis='VLBA_TL016B_split_lsrk.ms',spw='0:0~5,1:0~5',field='0,1', timerange='2022/02/21/06:14:00~2022/02/21/07:55:45.00', regridms=True,outframe='lsrk',datacolumn='all')
```




In [2]:
import graphviper

toolviper.utils.data.download(file="VLBA_TL016B_split.ms")

[2024-08-29 14:20:49,864]  WARNING  graphviper:  File exists: /home/fedemp/ws_xradio_pointing/venv_xradio_python_39/lib/python3.9/site-packages/graphviper/utils/data/.dropbox 
[2024-08-29 14:20:49,867]     INFO  graphviper:  Updating file metadata information ...  
 

 Download List         
 ────────────────────── 
  VLBA_TL016B_split.ms

VLBA_TL016B_split.ms.zip:   0%|          | 0.00/8.97M [00:00<?, ?iB/s]

## Convert to Processing Set

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

ms_file = "VLBA_TL016B_split.ms"

main_chunksize = {"frequency": 1, "time": 20}  # baseline, polarization
outfile = "VLBA_TL016B_split_lsrk.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

[2024-08-29 14:20:56,709]     INFO  graphviper:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBSERVATION_ID', 'FIELD_ID'] 
[2024-08-29 14:20:56,732]     INFO  graphviper:  Number of partitions: 4 
[2024-08-29 14:20:56,733]     INFO  graphviper:  OBSERVATION_ID [0], DDI [0], STATE [-1], FIELD [0], SCAN [0] 
[2024-08-29 14:20:57,070]  WARNING  graphviper:  Source_id is -1. No source information will be included in the field_and_source_xds. 
[2024-08-29 14:20:57,275]     INFO  graphviper:  OBSERVATION_ID [0], DDI [0], STATE [-1], FIELD [1], SCAN [0] 
[2024-08-29 14:20:57,637]  WARNING  graphviper:  Source_id is -1. No source information will be included in the field_and_source_xds. 
[2024-08-29 14:20:57,922]     INFO  graphviper:  OBSERVATION_ID [0], DDI [1], STATE [-1], FIELD [0], SCAN [0] 
[2024-08-29 14:20:58,452]  WARNING  graphviper:  Source_id is -1. No source information will be included in the field_and_source_xds. 
[2024-08-29 14:20:58,670]     INFO  graphviper:  OBSERV

## Processing Set

In [4]:
from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set(ps_store=outfile)
ps.summary()

,name,obs_mode,shape,polarization,scan_number,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
2,VLBA_TL016B_split_lsrk_0,[obs_0],"(200, 55, 6, 2)","[RR, LL]",[0],spw_0,[4C39.25_0],[Unknown],[],"[fk5, 9h27m03.01s, 39d02m20.85s]",5.004000e+09,5.006500e+09
3,VLBA_TL016B_split_lsrk_1,[obs_0],"(540, 55, 6, 2)","[RR, LL]",[0],spw_0,[J1154+6022_1],[Unknown],[],"[fk5, 11h54m04.54s, 60d22m20.82s]",5.004000e+09,5.006500e+09
0,VLBA_TL016B_split_lsrk_2,[obs_0],"(200, 55, 6, 2)","[RR, LL]",[0],spw_1,[4C39.25_0],[Unknown],[],"[fk5, 9h27m03.01s, 39d02m20.85s]",5.068000e+09,5.070500e+09
1,VLBA_TL016B_split_lsrk_3,[obs_0],"(540, 55, 6, 2)","[RR, LL]",[0],spw_1,[J1154+6022_1],[Unknown],[],"[fk5, 11h54m04.54s, 60d22m20.82s]",5.068000e+09,5.070500e+09


In [5]:
ps.keys()

dict_keys(['VLBA_TL016B_split_lsrk_2', 'VLBA_TL016B_split_lsrk_3', 'VLBA_TL016B_split_lsrk_0', 'VLBA_TL016B_split_lsrk_1'])

In [6]:
ps["VLBA_TL016B_split_lsrk_3"]

<xarray.Dataset> Size: 6MB
Dimensions:                     (time: 540, baseline_id: 55, frequency: 6,
                                 polarization: 2, uvw_label: 3)
Coordinates:
    baseline_antenna1_name      (baseline_id) <U2 440B dask.array<chunksize=(55,), meta=np.ndarray>
    baseline_antenna2_name      (baseline_id) <U2 440B dask.array<chunksize=(55,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 440B 0 1 2 3 ... 51 52 53 54
  * frequency                   (frequency) float64 48B 5.068e+09 ... 5.07e+09
  * polarization                (polarization) <U2 16B 'RR' 'LL'
    scan_number                 (time) int64 4kB dask.array<chunksize=(540,), meta=np.ndarray>
  * time                        (time) float64 4kB 1.645e+09 ... 1.645e+09
  * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 238kB dask.array<chunksize=(20, 55), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool 356kB dask.array<chunksize=(20, 55, 1, 2), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 238kB dask.array<chunksize=(20, 55), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 713kB dask.array<chunksize=(20, 55, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 3MB dask.array<chunksize=(20, 55, 1, 2), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 1MB dask.array<chunksize=(20, 55, 1, 2), meta=np.ndarray>
Attributes:
    data_groups:     {'base': {'flag': 'FLAG', 'uvw': 'UVW', 'visibility': 'V...
    partition_info:  {'field_name': ['J1154+6022_1'], 'line_name': [], 'num_l...
    type:            visibility
    weather_xds:     <xarray.Dataset> Size: 486kB\nDimensions:         (stati...
    antenna_xds:     <xarray.Dataset> Size: 116kB\nDimensions:               ...

In [8]:
ant_xds = ps["VLBA_TL016B_split_lsrk_3"].attrs["antenna_xds"]
ant_xds

<xarray.Dataset> Size: 116kB
Dimensions:                   (antenna_name: 10, cartesian_pos_label: 3,
                               receptor_label: 2, sky_dir_label: 2,
                               poly_term: 1, time_phase_cal: 141, tone_label: 2)
Coordinates:
  * antenna_name              (antenna_name) <U2 80B 'BR' 'FD' ... 'PT' 'SC'
  * cartesian_pos_label       (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
    gain_curve_type           (antenna_name) <U9 360B dask.array<chunksize=(10,), meta=np.ndarray>
    mount                     (antenna_name) <U6 240B dask.array<chunksize=(10,), meta=np.ndarray>
    polarization_type         (antenna_name, receptor_label) <U1 80B dask.array<chunksize=(10, 2), meta=np.ndarray>
  * receptor_label            (receptor_label) <U5 40B 'pol_0' 'pol_1'
  * sky_dir_label             (sky_dir_label) <U3 24B 'ra' 'dec'
    station                   (antenna_name) <U2 80B dask.array<chunksize=(10,), meta=np.ndarray>
  * time_phase_cal            (time_phase_cal) float64 1kB 1.645e+09 ... 1.64...
  * tone_label                (tone_label) <U6 48B 'freq_0' 'freq_1'
Dimensions without coordinates: poly_term
Data variables:
    ANTENNA_DISH_DIAMETER     (antenna_name) float64 80B dask.array<chunksize=(10,), meta=np.ndarray>
    ANTENNA_FEED_OFFSET       (antenna_name, cartesian_pos_label) float64 240B dask.array<chunksize=(10, 3), meta=np.ndarray>
    ANTENNA_POSITION          (antenna_name, cartesian_pos_label) float64 240B dask.array<chunksize=(10, 3), meta=np.ndarray>
    BEAM_OFFSET               (antenna_name, receptor_label, sky_dir_label) float64 320B dask.array<chunksize=(10, 2, 2), meta=np.ndarray>
    GAIN_CURVE                (antenna_name, receptor_label, poly_term) float32 80B dask.array<chunksize=(10, 2, 1), meta=np.ndarray>
    GAIN_CURVE_INTERVAL       (antenna_name) float64 80B dask.array<chunksize=(10,), meta=np.ndarray>
    GAIN_CURVE_SENSITIVITY    (antenna_name, receptor_label) float32 80B dask.array<chunksize=(10, 2), meta=np.ndarray>
    PHASE_CAL                 (antenna_name, time_phase_cal, receptor_label, tone_label) complex64 45kB dask.array<chunksize=(10, 141, 2, 2), meta=np.ndarray>
    PHASE_CAL_CABLE_CAL       (antenna_name, time_phase_cal) float64 11kB dask.array<chunksize=(10, 141), meta=np.ndarray>
    PHASE_CAL_INTERVAL        (antenna_name, time_phase_cal) float64 11kB dask.array<chunksize=(10, 141), meta=np.ndarray>
    PHASE_CAL_TONE_FREQUENCY  (antenna_name, time_phase_cal, receptor_label, tone_label) float64 45kB dask.array<chunksize=(10, 141, 2, 2), meta=np.ndarray>
    RECEPTOR_ANGLE            (antenna_name, receptor_label) float64 160B dask.array<chunksize=(10, 2), meta=np.ndarray>
Attributes:
    overall_telescope_name:  VLBA
    relocatable_antennas:    False
    type:                    antenna

In [9]:
ant_xds.ANTENNA_POSITION

<xarray.DataArray 'ANTENNA_POSITION' (antenna_name: 10, cartesian_pos_label: 3)> Size: 240B
dask.array<open_dataset-ANTENNA_POSITION, shape=(10, 3), dtype=float64, chunksize=(10, 3), chunktype=numpy.ndarray>
Coordinates:
  * antenna_name         (antenna_name) <U2 80B 'BR' 'FD' 'HN' ... 'PT' 'SC'
  * cartesian_pos_label  (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
    gain_curve_type      (antenna_name) <U9 360B dask.array<chunksize=(10,), meta=np.ndarray>
    mount                (antenna_name) <U6 240B dask.array<chunksize=(10,), meta=np.ndarray>
    station              (antenna_name) <U2 80B dask.array<chunksize=(10,), meta=np.ndarray>
Attributes:
    coordinate_system:  geocentric
    ellipsoid:          GRS80
    type:               earth_location
    units:              ['m', 'm', 'm']

In [12]:
ant_xds.BEAM_OFFSET

<xarray.DataArray 'BEAM_OFFSET' (antenna_name: 10, receptor_label: 2,
                                 sky_dir_label: 2)> Size: 320B
dask.array<open_dataset-BEAM_OFFSET, shape=(10, 2, 2), dtype=float64, chunksize=(10, 2, 2), chunktype=numpy.ndarray>
Coordinates:
  * antenna_name       (antenna_name) <U2 80B 'BR' 'FD' 'HN' ... 'OV' 'PT' 'SC'
    gain_curve_type    (antenna_name) <U9 360B dask.array<chunksize=(10,), meta=np.ndarray>
    mount              (antenna_name) <U6 240B dask.array<chunksize=(10,), meta=np.ndarray>
    polarization_type  (antenna_name, receptor_label) <U1 80B dask.array<chunksize=(10, 2), meta=np.ndarray>
  * receptor_label     (receptor_label) <U5 40B 'pol_0' 'pol_1'
  * sky_dir_label      (sky_dir_label) <U3 24B 'ra' 'dec'
    station            (antenna_name) <U2 80B dask.array<chunksize=(10,), meta=np.ndarray>
Attributes:
    frame:    fk5
    type:     sky_coord
    units:    ['rad', 'rad']